In [ ]:
#2:24:04
from ultralytics import YOLO
import torch
import cv2
import cvzone
import math
import numpy as np
from sort import *

model = YOLO("yolo-weights/yolov8n.pt")

cap =  cv2.VideoCapture("resources/yolo/people.mp4") # For Video


classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

mask = cv2.imread("resources/yolo/mask-1.png")

#Tracking
tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)

limits = [400,297,673,297]
totalCount = []

while True:
    success, img = cap.read()
    imgRegion = cv2.bitwise_and(img,mask)

    imgGraphics = cv2.imread("resources/yolo/graphics-1.png",cv2.IMREAD_UNCHANGED)
    img = cvzone.overlayPNG(img,imgGraphics,(730,260))
    results = model(imgRegion,stream=True)

    detections = np.empty((0,5))
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # cvzone bounding boxes
            x1,y1,x2,y2 = box.xyxy[0]
            x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
            w,h = x2-x1,y2-y1
            
            #Confidence
            conf = math.ceil(box.conf[0]*100)/100
            #Class Name
            cls = int(box.cls[0])
            currentClass = classNames[cls]

            if currentClass == "person" and conf >0.3:
                # cvzone.putTextRect(img,f'{currentClass} {conf}',(max(0,x1),max(35,y1)),scale=0.6,thickness=1,offset=3)
                #cvzone.cornerRect(img,(x1,y1,w,h),l=9,rt=5)
                currentArray = np.array([x1,y1,x2,y2,conf])
                detections = np.vstack((detections,currentArray))
    
    resultsTracker = tracker.update(detections)

    cv2.line(img,(limits[0],limits[1]),(limits[2],limits[3]),(0,0,255),5)

    for result in resultsTracker:
        x1, y1, x2, y2, id = result
        x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
        # print(result)
        w,h = x2-x1,y2-y1
        cvzone.cornerRect(img,(x1,y1,w,h),l=9,rt=2,colorR=(255,0,255))
        cvzone.putTextRect(img,f'{int(id)}',(max(0,x1),max(35,y1)),scale=2,thickness=3,offset=10)

        cx,cy = x1+w//2,y1+h//2
        cv2.circle(img,(cx,cy),5,(255,0,255),cv2.FILLED)

        if limits[0] < cx < limits[2] and limits[1]-15 < cy < limits[1]+15:
            if totalCount.count(id) ==0:
                totalCount.append(id)
                cv2.line(img,(limits[0],limits[1]),(limits[2],limits[3]),(0,255,0),5)
            
    # cvzone.putTextRect(img,f' Count: {len(totalCount)}',(50,50))
    cv2.putText(img,str(len(totalCount)),(255,100),cv2.FONT_HERSHEY_PLAIN,5,(50,50,255),8)
    
    cv2.imshow("Image",img)
    # cv2.imshow("ImageRegion",imgRegion)
    cv2.waitKey(1)






0: 384x640 5 cars, 65.2ms
Speed: 4.2ms preprocess, 65.2ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 70.0ms
Speed: 1.7ms preprocess, 70.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 60.6ms
Speed: 1.3ms preprocess, 60.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 55.5ms
Speed: 1.8ms preprocess, 55.5ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 58.7ms
Speed: 1.4ms preprocess, 58.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 55.7ms
Speed: 1.3ms preprocess, 55.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 53.2ms
Speed: 1.2ms preprocess, 53.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 54.9ms
Speed: 1.3ms preprocess, 54.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x